# Example Notebook for annotator package
## Scientific Software Center, I. S. Ulusoy, C. Delavier, Heidelberg University
*January 2022*

In the following the basic functionalities of the package are introduced. We will load basic text in English and German and annotate it using the available features.

### Import the modules of the annotator package

In [ ]:
import base as be
import mspacy as msp
import mstanza as sa

# General input

The input is passed to the package using a dictionary (json-file). Later, this will be hidden in the user interface. For now, you will load the default dictionary which has all options pre-set to default values, and then replace the options that you specify for your desired processing.

### Read the default dictionary - do not change

In [ ]:
# read in input.json
default_dict = be.prepare_run.load_input_dict("input")

The main input dictionary contains three important parameters:
```
    "input": "./test/test_files/example_en.txt",
    "output": "out/output_en",
    "tool": "spacy",
```
You can print these using:

In [ ]:
print(default_dict["input"])
print(default_dict["output"])
print(default_dict["tool"])

These tell the programm that the data we want to annotate is stored in `example_en.txt` with path to the file `./test/test_files/`, that we want to output to a file which we can identify as `output_en` in the folder `out` and that we want to use the tool `spacy` to annotate the data.

### Modify the keys for your specific run - do change

You can now modify the keys to specify a different input file, output file, and selected tool as so:

In [ ]:
# first make a copy of the dictionary for your run
mydict = default_dict
# now you need to set your parameters
# change the value of the key on the right hand of the "="
mydict["input"] = "./test/test_files/example_en.txt"
# change the value of the key on the right hand of the "="
mydict["output"] = "output_en"
mydict["tool"] = "spacy"  # or "stanza" - so far, only spacy and stanza are implemented

Please note the "" around the keys - these are essential as the values are passed as string and should not be removed!

### Read in the input text to be processed as raw text - do not change

In [ ]:
# read in the raw text
data = be.prepare_run.get_text(mydict["input"])

You can print the text as so:

In [ ]:
print(data)

You may also directly copy and paste text here - take care that it is surrounded by double quotes again:

In [ ]:
data = "This is my text. I like it better this way."

# Process the text using SpaCy
More information about SpaCy is found [here](https://spacy.io/). Generally, SpaCy supports [these languages](https://spacy.io/usage/models), but at the moment only English and German are available in the annotator package. We will add more languages based on your requests - so please get in touch!

## TL;DR - the essence
You need to specify which language you would like to process. It is also good to specify the processing options, like tokenization, part-of-speech, lemma, etc., although if not specified the package will select all that are available for the language. There is currently a restriction: The output that is generated in the end and passed to cwb can only contain one or several of these options: sentencize, tokenize, part-of-speech, lemma. All other options do get processed but are not written to the file yet. Here we need some more feedback on the format that is required for cwb.

In [ ]:
# check which language has been selected
print(mydict["spacy_dict"]["lang"])

You can change the selected language as so:

In [ ]:
mydict["spacy_dict"]["lang"] = "en"
# currently, only "en" and "de" are available

In [ ]:
# find out which model is being used
print(mydict["spacy_dict"]["model"])

You will be able to change the model, if another one has been downloaded. At the moment, only `en_core_web_md` and `de_core_news_md` are available. We will add more upon request, so please get in touch!

Now select the processors that you would like to use: For the default English pipeline, the available options are `tok2vec, senter, tagger, parser, attribute_ruler, lemmatizer, ner`, where the first two options are required for tokenization, and the other options are: [Dependency parser](https://spacy.io/api/dependencyparser), POS-tagging via the [attribute ruler](https://spacy.io/api/attributeruler), [lemma](https://spacy.io/api/lemmatizer), and [named-entity recognition](https://spacy.io/api/entityrecognizer). 

In [ ]:
# check which processors have been selected
print(mydict["spacy_dict"]["processors"])

In [ ]:
# change these if you like as so
mydict["spacy_dict"][
    "processors"
] = "tok2vec, senter, tagger, attribute_ruler, lemmatizer"

## In more detail for the interested user

There will be more tools to chose from, but for simplicity their configurations have been stripped for now. The spacy specific config is found in the `"spacy_dict"` section of the main input dictionary. Here we find the parameters we can tell spacy to enable it to annotate the data. The entries do usually come with a comment explaining what the parameters do. Lets look through the ones we set up in input.json:
```
    "model": false,
```
Here we can specify a model spacy should use to annotate the text if we want to. We leave it to false for now though.
```
    "lang": "en",
```
Here we specify that the language of the data we want to annotate is english. Since we didn't specify a model this information will be needed to chose one for us.
```
    "text_type": "news",
```
We specify what kind of text we want to annotate in order to chose an appropriate model for the task. This does currently only support "news" for english. The setup we chose here will lead to the usage of the model `en_core_web_md`.
```
    "processors": "tok2vec, senter, tagger, parser, attribute_ruler, lemmatizer, ner",
```
Here we specifiy the processors for the pipeline we will apply to our data. This will define what kind of annotations we get in the end, as well as potentionally impacting performance of the pipeline. The availability of specific processors is dependent upon the selected model. The module checks if all requested processors are available before trying to load them and should tell us if there is a problem. Available pipeline components for pretrained `spacy models` can be found in the [spacy models documentation](https://spacy.io/models).

The remaining entries are not immediatly important for this example and are all set to their default values. Especially the `"config"` parameter and its contents are defined for a given pretrained model in it's config.cfg file, changing this is not recommended unless you really know what you are doing.

Next we would load the tool as specified by the .json. In this case we would load the spacy pipeline from the `mspacy` module. We are told what components we load for our pipeline and which model we are using. In our case we load the models `en_core_web_md` and `de_core_news_md` with all their pipeline components. The function calls below create `spacy_pipe` objects which we can than apply to data.

## Load the SpaCy pipeline and process the text

In [ ]:
# load the pipeline from the config
pipe = msp.spacy_pipe(mydict)

After doing this we only have to apply the pipeline to the data we read in earlier. For this we use the `apply_to` function of the `spacy_pipe` object, this generates the annotated `spacy.Doc` which is stored in our `spacy_pipe` object.

In [ ]:
# apply pipeline to data
annotated = pipe.apply_to(data)

To extract the results of the pipeline  we can easily pass the results to a .vrt file using the output name defined in the .json. This is done by using the `pass_results` function build into the `spacy_pipe` object. Doing it this way, we also directly encode our results for `CWB`.

In [ ]:
# get the annotated .vrt and pass to cwb
annotated.pass_results()

Loading the pipeline, applying it and passing the results can be done conveniently in one line:

In [ ]:
# example for english data
msp.spacy_pipe(mydict).apply_to(data).pass_results()

# is equivalent to the above
# pipe_en = msp.spacy_pipe(dict_en)
# annotated_en = pipe_en.apply_to(data_en)
# annotated_en.pass_results()

# Using a different tool
In this section we will be using a different tool to annotate our data. The only other available tool at this moment is `stanza`. Looking at `example_stanza.json` we see that we now have set
```
    "tool": "stanza",
```
which woul indicate to the program that we do indeed want to use `stanza`. As data we will again use `example_en.txt`.

In [ ]:
# load the dict
dict_sa = be.prepare_run.load_input_dict("example_stanza")

outname = dict_sa["output"]

In [ ]:
# get relevant part of dict for stanza
stanza_dict = dict_sa["stanza_dict"]

# remove the comments
stanza_dict = be.prepare_run.update_dict(stanza_dict)

To use the processors requested in `example_stanza.json` in we have to activate them for `stanza`.

In [ ]:
# activate processors
stanza_dict = be.prepare_run.activate_procs(stanza_dict, "stanza_")

In [ ]:
# we still have the data from before, but will just get it again
data_en = be.prepare_run.get_text("example_en.txt")

After setting up the dictionary we can create a `mstanza_pipeline` object from the `mstanza` module. Notice that you might have to change the path set in `"dir"` to point to your installation of the `stanza` models.

In [ ]:
# if you want to download the model to the needed path execute the code below.
# Note that the model has a size of several hundred MB.

# from stanza import download
# download("en", model_dir = "test/models/")

In [ ]:
# initialize the pipeline with the dict
stanza_pipe = sa.mstanza_pipeline(stanza_dict)

# to get the working pipeline we have to use the inbuilt initialize function
stanza_pipe.init_pipeline()

We can then apply the pipeline to our data through the `process_text` function of the `mstanza_pipeline` object.

In [ ]:
# apply pipeline to data
results = stanza_pipe.process_text(data_en)

To write the results to a file and export to `CWB` we can then use the `mstanza_pipeline.postprocess` function. 

In [ ]:
stanza_pipe.postprocess(outname)